In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE


# Load the dataset
df = pd.read_csv('../train.csv')

# Create a mapping from metaphorID to the actual metaphor words
metaphor_mapping = {
    0: 'road', 1: 'candle', 2: 'light', 3: 'spice', 4: 'ride', 5: 'train', 6: 'boat'
}
df['metaphor_word'] = df['metaphorID'].map(metaphor_mapping)

# Convert the 'label_boolean' column to integers (True to 1, False to 0)
df['label_boolean'] = df['label_boolean'].astype(int)

# Splitting the dataset into training and testing sets first
X_train_raw, X_test_raw, y_train, y_test = train_test_split(df[['text', 'metaphor_word']], df['label_boolean'], test_size=0.2, random_state=42, stratify=df['label_boolean'])

# Feature extraction using TF-IDF for the text (fit on training data only)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_raw['text']).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw['text']).toarray()

# One-hot encode the 'metaphor_word' column
onehot_encoder = OneHotEncoder(sparse=False)
X_train_metaphor = onehot_encoder.fit_transform(X_train_raw[['metaphor_word']])
X_test_metaphor = onehot_encoder.transform(X_test_raw[['metaphor_word']])

# Combine TF-IDF features with one-hot encoded metaphor_word features
X_train = np.hstack((X_train_tfidf, X_train_metaphor))
X_test = np.hstack((X_test_tfidf, X_test_metaphor))

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define models with best parameters
models = [
    ("Random Forest", RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced')),
    ("AdaBoost", AdaBoostClassifier(learning_rate=0.01, n_estimators=200, random_state=42)),
    ("Gradient Boosting", GradientBoostingClassifier(learning_rate=0.1, max_depth=10, n_estimators=200, random_state=42)),
    ("Extra Trees", ExtraTreesClassifier(n_estimators=100, random_state=42)),
    ("XGBoost", XGBClassifier(learning_rate=0.01, max_depth=6, n_estimators=300, random_state=42, use_label_encoder=False, eval_metric='logloss'))
]

# Train and evaluate each model
for name, model in models:
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    print(f"Model: {name}")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}\n")


/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Model: Random Forest
              precision    recall  f1-score   support

           0       0.73      0.36      0.48        88
           1       0.83      0.96      0.89       286

    accuracy                           0.82       374
   macro avg       0.78      0.66      0.69       374
weighted avg       0.81      0.82      0.79       374

Accuracy: 0.8181818181818182

Model: AdaBoost
              precision    recall  f1-score   support

           0       0.49      0.39      0.43        88
           1       0.82      0.87      0.85       286

    accuracy                           0.76       374
   macro avg       0.65      0.63      0.64       374
weighted avg       0.74      0.76      0.75       374

Accuracy: 0.7593582887700535

Model: Gradient Boosting
              precision    recall  f1-score   support

           0       0.74      0.58      0.65        88
           1       0.88      0.94      0.91       286

    accuracy                           0.85       374
   mac

/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Model: XGBoost
              precision    recall  f1-score   support

           0       0.66      0.57      0.61        88
           1       0.87      0.91      0.89       286

    accuracy                           0.83       374
   macro avg       0.77      0.74      0.75       374
weighted avg       0.82      0.83      0.82       374

Accuracy: 0.8288770053475936



In [2]:
# Initialize an array to store the sum of predicted probabilities
prob_sum = np.zeros((len(y_test), 2))  # Assuming binary classification

# Train and evaluate each model, and accumulate probabilities
for name, model in models:
    model.fit(X_train_resampled, y_train_resampled)
    probs = model.predict_proba(X_test)
    prob_sum += probs

    y_pred = model.predict(X_test)
    print(f"Model: {name}")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}\n")

# Compute the average probabilities
avg_prob = prob_sum / len(models)

# Determine final predictions based on the highest probability
final_predictions = np.argmax(avg_prob, axis=1)

# Evaluate the final ensemble model
print("Ensemble Model Performance:")
print(classification_report(y_test, final_predictions))
print(f"Accuracy: {accuracy_score(y_test, final_predictions)}\n")

Model: Random Forest
              precision    recall  f1-score   support

           0       0.73      0.36      0.48        88
           1       0.83      0.96      0.89       286

    accuracy                           0.82       374
   macro avg       0.78      0.66      0.69       374
weighted avg       0.81      0.82      0.79       374

Accuracy: 0.8181818181818182

Model: AdaBoost
              precision    recall  f1-score   support

           0       0.49      0.39      0.43        88
           1       0.82      0.87      0.85       286

    accuracy                           0.76       374
   macro avg       0.65      0.63      0.64       374
weighted avg       0.74      0.76      0.75       374

Accuracy: 0.7593582887700535

Model: Gradient Boosting
              precision    recall  f1-score   support

           0       0.74      0.58      0.65        88
           1       0.88      0.94      0.91       286

    accuracy                           0.85       374
   mac

/Users/ayushlodha/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Model: XGBoost
              precision    recall  f1-score   support

           0       0.66      0.57      0.61        88
           1       0.87      0.91      0.89       286

    accuracy                           0.83       374
   macro avg       0.77      0.74      0.75       374
weighted avg       0.82      0.83      0.82       374

Accuracy: 0.8288770053475936

Ensemble Model Performance:
              precision    recall  f1-score   support

           0       0.74      0.52      0.61        88
           1       0.87      0.94      0.90       286

    accuracy                           0.84       374
   macro avg       0.80      0.73      0.76       374
weighted avg       0.84      0.84      0.83       374

Accuracy: 0.8449197860962567

